In [1]:
import pickle
from pathlib import Path

# Ruta al archivo results.pkl (ajústala según corresponda)
ruta = Path(r"E:\Aplicaciones\Nuevo_PRG\Compara_PRG\data\results\results_20250723_01.pkl")

# Cargar el archivo
with open(ruta, "rb") as f:
    data = pickle.load(f)

# Ver tipo de objeto cargado
print("Tipo:", type(data))

# Si es un diccionario, mostrar claves
if isinstance(data, dict):
    print("\nClaves principales:", list(data.keys())[:10])
    
    # Mostrar el tipo de los primeros valores
    for k, v in list(data.items())[:2]:
        print(f"\nClave: {k} → tipo: {type(v)}")
        if isinstance(v, dict):
            print("  Subclaves:", list(v.keys())[:10])

# Si es una lista, mostrar ejemplos
elif isinstance(data, list):
    print("\nEjemplo de elementos:", data[:5])

# En cualquier otro caso, mostrar contenido directo
else:
    print("\nContenido:", data)


Tipo: <class 'collections.defaultdict'>

Claves principales: ['PCP', 'PID1']

Clave: PCP → tipo: <class 'dict'>
  Subclaves: ['COTAS', 'CMG', 'GENTABLES', 'GENT', 'BESS', 'GENC']

Clave: PID1 → tipo: <class 'dict'>
  Subclaves: ['COTAS', 'CMG', 'GENTABLES', 'GENT', 'BESS', 'GENC']


In [3]:
data['PID1']['BESS']

Nombre_PLEXOS,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""SAE-ARICA""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,1.965024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,1.965024,0.0,0.0,0.0,0.0,0.0
"""SAE-DEL-DESIERTO""",0.0,0.0,0.0,0.0,0.0,-7.711431,0.0,0.0,-15.239295,-200.0,-200.0,-200.0,0.0,0.0,200.0,36.7817,159.480505,200.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-70.617183,-110.686382,-200.0,-200.0,-42.425851,0.0,200.0,200.0,141.517017,63.500516,0.0,0.0,0.0
"""SAE-NUEVA-IMPERIAL""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""SAE-CRCA-ALFALFAL-VR1""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""SAE-CRCA-ALFALFAL-VR2""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-49.24,-49.25,-49.25,-49.25,-0.01,0.0,49.25,49.25,39.859048,49.25,0.01,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""SAE-CRCA-PFV-SAN-ANDRES""",0.0,0.0,0.0,0.0,0.0,-19.295652,-23.008696,-23.452174,-23.217391,-23.33913,-23.817391,-22.452174,-16.417391,0.0,35.0,35.0,35.0,35.0,33.267327,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.808696,-22.53913,-23.069565,-23.365217,-23.678261,-24.426087,-23.182609,-18.930435,0.0,35.0,35.0,35.0,35.0,33.267327,0.0,0.0
"""SAE-CRCA-PFV-TAMAYA""",0.0,0.0,0.0,0.0,0.0,-13.939216,-59.696078,-59.529412,-59.862745,-61.058824,-62.5,-62.931373,-53.382353,0.0,78.0,78.0,78.0,78.0,78.0,30.291262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-6.684314,-60.901961,-59.666667,-58.921569,-60.156863,-63.137255,-64.705882,-58.72549,0.0,78.0,78.0,78.0,78.0,78.0,30.291262,0.0
"""SAE-CRCA-PFV-URIBE-SOLAR""",0.0,0.0,0.0,0.0,0.0,-0.009524,0.008707,0.0,0.0,0.0,-2.013271,-0.537206,-0.009524,0.0,2.340464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.06,-2.5,0.0,0.0,0.0,2.340464,0.0,0.0,0.0,0.0,0.0


In [1]:
import os, sys, re, clr, csv
from shutil import copyfile
import pandas as pd
import subprocess as sp
from os.path import dirname, join
from System.IO import SearchOption as SearchOption
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, Animation, ArtistAnimation
from datetime import datetime
import numpy as np
from System import *

sys.path.append('C:/Program Files/Energy Exemplar/PLEXOS 10.0 API/')
clr.AddReference('PLEXOS_NET.Core')
clr.AddReference('EEUTILITY')
clr.AddReference('EnergyExemplar.PLEXOS.Utility')

from PLEXOS_NET.Core import *
from EEUTILITY.Enums import *
from EnergyExemplar.PLEXOS.Utility.Enums import *

copy_file = r"C:\Users\maria.perez\Documents\PID\API_PLEXOS\DBSEN_PRGDIARIO_PID.xml"

if not os.path.exists(copy_file):
    print("❌ Archivo no encontrado")
    exit()

# Crear conexión al modelo
db = DatabaseCore()
db.DisplayAlerts = False
db.Connection(copy_file)
classes = db.FetchAllClassIds()

# Buscar IDs
collections = db.FetchAllCollectionIds()
properties  = db.FetchAllPropertyEnums()
attributes  = db.FetchAllAttributeEnums()
classes     = db.FetchAllClassIds()

df_final shape: (2313, 5)
        Tipo                      Linea        Nodo        Lat        Lon
0  LineaFrom  AJahuel110->Sauzal110_BP1  AJahuel110 -33.714834 -70.691174
1  LineaFrom       AJahuel154->Paine154  AJahuel154 -33.714834 -70.691174
2  LineaFrom  AJahuel154->Tuniche154_II  AJahuel154 -33.714834 -70.691174
3  LineaFrom     AJahuel220->AJahuel110  AJahuel220 -33.714834 -70.691174
4  LineaFrom     AJahuel220->AJahuel154  AJahuel220 -33.714834 -70.691174
5  LineaFrom        AJahuel220->Buin110  AJahuel220 -33.714834 -70.691174
6  LineaFrom       AJahuel220->Chena220  AJahuel220 -33.714834 -70.691174
7  LineaFrom   AJahuel220->PAltoCmpc110  AJahuel220 -33.714834 -70.691174
8  LineaFrom  AJahuel220->SantaMarta220  AJahuel220 -33.714834 -70.691174
9  LineaFrom     AJahuel500->AJahuel220  AJahuel500 -33.714834 -70.691174
